# 2.6.2. Particularización de la matriz de tensiones al caso bidimensional

|Quién | Correo | Fecha | Qué hizo |
| ---  | ---  | --- | ---      |
|Michael Heredia Pérez | <mherediap@unal.edu.co> | Julio 13, 2022       | Conversión de código de MAXIMA a Python|
|Michael Heredia Pérez | <mherediap@unal.edu.co> | Agosto 31, 2022      | Mejorando presentación                 |
|Diego Andrés Alvarez  | <daalvarez@unal.edu.co>  | Septiembre 12, 2022 | Mejorando comentarios                  |
|Juan Nicolás Ramírez  | <jnramirezg@unal.edu.co> | Febrero 2, 2023     |  Simplificando programa                |

$$
\newcommand{\ve}[1]{{\vec{\boldsymbol{#1}}}}
\newcommand{\ma}[1]{{\boldsymbol{#1}}}
\newcommand{\hve}[1]{{\,\hat{\!\boldsymbol{#1}}}}
$$

In [1]:
from sympy import init_printing, latex, symbols, sin, cos, Matrix, trigsimp, linear_eq_to_matrix, fu

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

Se definen las variables simbólicas necesarias.

In [2]:
sx, sy, txy, t = symbols("sigma_x, sigma_y, tau_xy, theta")

Se define la matriz de transformación $\ma{T}$:

$$ \ma{T} = 
\begin{bmatrix}
\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} & 0\\
\sin{\left(\theta \right)} & \cos{\left(\theta \right)} & 0\\
0 & 0 & 1
\end{bmatrix}
$$

In [3]:
T = Matrix([[cos(t), -sin(t),  0],
            [sin(t),  cos(t),  0],
            [     0,       0,  1]])

imprimir(r"\ma{T} = ", T)

<IPython.core.display.Math object>

Se define la matriz de tensiones en coordenadas rectangulares $\ma{\sigma}$:

$$
\ma{\sigma} = 
\begin{bmatrix}
	\sigma_x  & \tau_{xy} & \tau_{xz}\\
	\tau_{xy} & \sigma_y  & \tau_{yz}\\
	\tau_{xz} & \tau_{yz} & \sigma_z
\end{bmatrix}
$$        

In [4]:
sigma = Matrix([[ sx, txy,  0],
                [txy,  sy,  0],
                [  0,   0,  0]])

imprimir(r"\ma{\sigma} = ", sigma)

<IPython.core.display.Math object>

Se calcula la matriz de tensiones $\ma{\sigma}'$ en el sistema de coordenadas especificadas por los vectores definidos en la matriz $\ma{T}$ usando la ecuación:

$$
\ma{\sigma}' = \ma{T}^T\ma{\sigma}\ma{T}
$$

In [5]:
sigmaP = T.T*sigma*T
# sp.Transpose(R) es lo mismo que R.T

No contamos con la función ```trigreduce()``` de MAXIMA, pero podemos llegar a un resultado similar, poco simplificado utilizando ``trigsimp()``:

In [6]:
# trigsimp() escribe la expresión en términos del ángulo doble
sigmaP = trigsimp(sigmaP)
imprimir(r"\ma{\sigma'} = ", sigmaP)

<IPython.core.display.Math object>

Se extraen los términos de la matriz de tensiones $\ma{\sigma'}$

In [7]:
sxp   = sigmaP[0,0]  # elemento 1,1 de la matriz sigmaP
syp   = sigmaP[1,1]  # elemento 2,2 de la matriz sigmaP
txpyp = sigmaP[0,1]  # elemento 1,2 de la matriz sigmaP 

In [8]:
imprimir(r"\sigma_{x'}(\theta) = ", sxp)

<IPython.core.display.Math object>

In [9]:
imprimir(r"\sigma_{y'}(\theta) = ", syp)

<IPython.core.display.Math object>

In [10]:
imprimir(r"\tau_{x'y'}(\theta) = ", txpyp)

<IPython.core.display.Math object>

Las tres ecuaciones anteriores las podemos expresar de forma matricial como:

$$
\begin{pmatrix}
\sigma_{x'}\\
\sigma_{y'}\\
\tau_{x'y'}
\end{pmatrix}
=
\underbrace{%
   \begin{pmatrix}
   \cos^2\theta           & \sin^2\theta          &  2\sin\theta\cos\theta \\
   \sin^2\theta           & \cos^2\theta          & -2\sin\theta\cos\theta \\
   -\sin\theta\cos\theta  & \sin\theta\cos\theta  & \cos^2\theta - \sin^2\theta
   \end{pmatrix}}_{\ma{T}_{\ve{\sigma},\text{2D}}}
\begin{pmatrix}
\sigma_x\\
\sigma_y\\
\tau_{xy}
\end{pmatrix}.
$$

Por lo tanto, usamos la función ```linear_eq_to_matrix()``` para encontrar la matriz $\ma{T}_{\ve{\sigma},\text{2D}}$:

In [11]:
Tsigma_2D = trigsimp(linear_eq_to_matrix([sxp, syp, txpyp], [sx, sy, txy])[0])
imprimir(r"\ma{T}_{\ve{\sigma},\text{2D}} = ", Tsigma_2D)

<IPython.core.display.Math object>

En Python tenemos los [algoritmos de Fu et. al.](https://docs.sympy.org/latest/modules/simplify/fu.html) para llevar a cabo las equivalencias trigonométricas. La función ```TR11``` nos ayuda a reducir las expresiones en función del ángulo sencillo:

In [12]:
from sympy.simplify.fu import TR11
imprimir(r"\ma{T}_{\ve{\sigma},\text{2D}} = ", TR11(Tsigma_2D))

<IPython.core.display.Math object>